<a href="https://colab.research.google.com/github/yubin-xing-usask/stance_markers/blob/main/Search_StanceMarkers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [20]:
# a function of removing namespace in XML files

def remove_namespace(doc, namespace):
    """Remove namespace in the passed document in place."""
    ns = u'{%s}' % namespace
    nsl = len(ns)
    for elem in doc.getiterator():
        if elem.tag.startswith(ns):
            elem.tag = elem.tag[nsl:]

In [ ]:
# a function finding file locations

import os

def location_xml(inpath):
  """
  This function input a path that stores all xml RAs, 
  returns the xml file location
  """
  path_list = []
  file_names = os.listdir(inpath)
  for name in file_names:
    if name.endswith(".xml"):
      file_location = str(inpath + "/" + name)
      path_list.append(file_location)
  return path_list

# location_xml('drive/MyDrive/Mitacs_project/Citation/MasterList/RAs_Engineering')

In [ ]:
# start extracting sentences with a ref tag
import xml.etree.ElementTree as ET
import re
import spacy

# save paragraphs with refs into a list
p_list = []
for f in location_xml('drive/MyDrive/Mitacs_project/Citation/MasterList/RAs_Engineering'):
  tree = ET.parse(f, parser = ET.XMLParser(encoding = 'utf-8'))
  root = tree.getroot()
  remove_namespace(root, u'http://www.tei-c.org/ns/1.0')
  
  for child in root.findall(".//ref[@type='bibr'].."): 
    # this line might need to be modified by using this xpath: ".//p" to locate all paragraphs
    st=child.itertext()
    childlist=list(st)
    seperator=", "
    seperator.join(childlist)
    p_list.append(childlist) # the ref and pargraph strings are seperated with ","
    # this step might need to be optimized

# save all text of these paragraphs into a list, join paragraph and reference strings together
plist_final=[]
for li in p_list:
    s=""
    lil=s.join(li)
    plist_final.append(lil)
# print(plist_final)
# now we have a full list with all body texts which contains all references (i.e. citations)


In [ ]:
# this step is to use regex to locate all necessary citations




In [ ]:
# with the locations of citations, find the stance markers, including:
# Reporting verbs, boosters, hedges, attitude markers, others.



In [ ]:
# !!! keep this cell
# test 6 remove the ref tag from Grobid and use spacy tools to tokenize the sentences, 
# then use regular expression to find the reference tags
import xml.etree.ElementTree as ET
import re
import spacy

# remove the namespace 
# looks like this step is not necessary anymore if using regex to search for citations
# but this function might be useful for future projects
def remove_namespace(doc, namespace):
    """Remove namespace in the passed document in place."""
    ns = u'{%s}' % namespace
    nsl = len(ns)
    for elem in doc.getiterator():
        if elem.tag.startswith(ns):
            elem.tag = elem.tag[nsl:]

# parse the XML file into an element Tree
tree = ET.parse('drive/MyDrive/Mitacs_project/Citation/ELS_HS_AMP_2015_V2_2.tei.xml', parser = ET.XMLParser(encoding = 'utf-8'))
root = tree.getroot()
remove_namespace(root, u'http://www.tei-c.org/ns/1.0')

# save paragraphs with refs into a list
p_list = []
for child in root.findall(".//ref[@type='bibr'].."): # this line might need to be modified by using this xpath: ".//p" to locate all paragraphs
    st=child.itertext()
    childlist=list(st)
    seperator=", "
    seperator.join(childlist)
    p_list.append(childlist)

# save all text of these paragraphs into a list, join paragraph and reference strings together
list1=[]
for li in p_list:
    s=""
    lil=s.join(li)
    list1.append(lil)

# tokenize every sentence with spacy
list_sent=[]
for each_p in list1:
    nlp = spacy.load("en_core_web_sm") 
    doc = nlp(each_p)
    for sent in doc.sents:
        list_sent.append(sent.text)

# use regex find the references and save them into a single list for the next analysis

refIN_sent=[]
for each_sen in list_sent:
    author = "(?:[A-Z][A-Za-z'`-]+)"
    etal = "(?:et al.?)"
    additional = "(?:,? (?:(?:and |& )?" + author + "|" + etal + "))"
    year_num = "(?:19|20)[0-9][0-9]"
    page_num = "(?:, p.? [0-9]+)?"  # Always optional
    year = "(?:, *"+year_num+page_num+"| *\("+year_num+page_num+"\))"
    regex = "(" + author + additional+"*" + year + ")"
    matches = re.search(regex, each_sen)
    if matches != None:
        refIN_sent.append(each_sen)

# annotate the sentences with a tag to identify it is intergral or non-integral citation

ref_tags = refIN_sent
have_tags = []
for element in ref_tags:
    year_paren="""\((?:19|20|)[0-9][0-9]\)"""
    yearmatches = re.search(year_paren, element)
    if yearmatches != None:
        element = "<int>" + element + "</int>"
    else:
        element = "<non_int>" + element + "</non_int>"
    have_tags.append(element)
print(have_tags)

# use spacy to tokenize these sentences and save it into an array or CSV
for se in refIN_sent:
    doc_se = nlp(se)
    #for token_se in doc_se:
       # print(token_se.text,",", token_se.lemma_, ",", token_se.pos_, ",", token_se.tag_,  ",", token_se.dep_,
       #      ",", token_se.shape_,  ",", token_se.is_alpha,  ",", token_se.is_stop)

# don't forget to count the number of refs extracted by using regex, and compare it with Grobid results
# see which one is better

# New Section

In [ ]:
#regex patterns to search for citations
author = "(?:[A-Z][A-Za-z'`-]+)"
etal = "(?:et al.?)"
additional = "(?:,? (?:(?:and |& )?" + author + "|" + etal + "))"
year_num = "(?:19|20)[0-9][0-9]"
page_num = "(?:, p.? [0-9]+)?"  # Always optional
year = "(?:, *"+year_num+page_num+"| *\("+year_num+page_num+"\))"
regex = "(" + author + additional+"*" + year + ")"

matches = re.findall(regex, text)

In [ ]:
# remove a namespace from XML docs
import xml.etree.ElementTree as ET
def remove_namespace(doc, namespace):
    """Remove namespace in the passed document in place."""
    ns = u'{%s}' % namespace
    nsl = len(ns)
    for elem in doc.getiterator():
        if elem.tag.startswith(ns):
            elem.tag = elem.tag[nsl:]

metadata = '/Users/user1/Desktop/Python/metadata.xml'
tree = ET.parse(metadata)
root = tree.getroot()

remove_namespace(root, u'http://apple.com/itunes/importer')
tree.write('/Users/user1/Desktop/Python/done.xml',
       pretty_print=True, xml_declaration=True, encoding='UTF-8')